In [15]:
import pandas as pd
import numpy as np

In [16]:
sub = pd.read_csv('./ncrna_subset.csv')
# remove newlines and convert to lowercase
sub['sequence'] = sub['sequence'].map(lambda x: str.replace(x, '\n', '').lower())
sub.head()

,accession,sequence,type
0,CM008178.2/13519367-13518782,gttcccgcgcccggcccggagcgcgcctctgccgaggaccttgacc...,lncRNA
1,WEIX01007000.1/95142-95341,ctaggagaaccgaaaacagttctagactgacctgcggttttatagc...,lncRNA
2,NIFN02008677.1/26758913-26758776,atcgccagtacctactgcaacatctttcctccctacacagcgactc...,lncRNA
3,JH126708.1/1088160-1088311,ctcctttttctgcttgatcctgtagtgaagtcagaatcctgtgttt...,lncRNA
4,NC_032656.1/46729700-46729828,tctgccccctccccatgctctccctctttcctctcttcccctttca...,lncRNA


There are more than 4 unique characters (A,C,G,T) in some sequences due to ambiguous base calls from sequencing process:

In [17]:
bases_per_sequence = sub['sequence'].map(lambda x: ''.join(sorted(set(x))))
all_bases = set(''.join(bases_per_sequence.unique()))
n = len(all_bases)
print(all_bases)
print(n)

{'n', 's', 'r', 't', 'b', 'y', 'w', 'u', 'c', 'v', 'd', 'h', 'm', 'a', 'g', 'k'}
16


I decided to remove entries with sequences that contain bases other than A, G, C, or T, since there weren't that many. This will simplify the CGR a lot because we'll only need 4 vertices.

In [18]:
sub['unique_bases'] = sub['sequence'].map(lambda x: ''.join(sorted(set(x))))
acgt_only = sub.loc[sub['unique_bases'] == 'acgt',:]
acgt_only = acgt_only.drop(columns=['unique_bases'])

Setting positions of each vertex:

In [19]:
positions = {
    'a': (-1., 1.),
    't': (1., 1.),
    'g': (1., -1.),
    'c': (-1., -1.)
}

Below is the function used to calculate the CGR of each sequence:

In [20]:
def make_cgr(seq, sf, base_positions):
    # start with random position
    # CGR will approach about the same position once whole sequence is processed, no matter the starting position,
    # But randomly selecting start position might prevent points from overlapping
    [x,y] = ((np.random.rand(2)) * 2) - 1
    for c in seq:
        x = x + (sf * (base_positions[c][0] - x))
        y = y + (sf * (base_positions[c][1]- y))
    return (x,y)

In [21]:
acgt_only.loc[:, ['cgr_x', 'cgr_y']] = [0., 0.]
new_sf = 0.5
for i in acgt_only.index:
    cgr = make_cgr(acgt_only.loc[i,'sequence'], new_sf, positions)
    acgt_only.loc[i,['cgr_x', 'cgr_y']] = [cgr[0], cgr[1]]

In [22]:
acgt_only.head()

,accession,sequence,type,cgr_x,cgr_y
0,CM008178.2/13519367-13518782,gttcccgcgcccggcccggagcgcgcctctgccgaggaccttgacc...,lncRNA,0.275957,-0.338453
1,WEIX01007000.1/95142-95341,ctaggagaaccgaaaacagttctagactgacctgcggttttatagc...,lncRNA,0.662054,-0.419805
2,NIFN02008677.1/26758913-26758776,atcgccagtacctactgcaacatctttcctccctacacagcgactc...,lncRNA,0.749801,-0.671359
3,JH126708.1/1088160-1088311,ctcctttttctgcttgatcctgtagtgaagtcagaatcctgtgttt...,lncRNA,-0.296876,0.298390
4,NC_032656.1/46729700-46729828,tctgccccctccccatgctctccctctttcctctcttcccctttca...,lncRNA,-0.206335,0.662376


Adding sequence length and % GC content as columns:

In [23]:
acgt_only['sequence length'] = acgt_only['sequence'].map(len)
acgt_only['GC content'] = ((acgt_only['sequence'].str.count('g') + acgt_only['sequence'].str.count('c')) / acgt_only['sequence length']) * 100

In [25]:
acgt_only.to_csv('cgr_subset.csv', index=False)